<a href="https://colab.research.google.com/github/Kyanji/Security/blob/master/CNN_on_malware.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Conversion from data to pixel

In [0]:
def ConvPixel(FVec, xp, yp, A, B, base=1):
    n = len(FVec)
    M = np.ones([int(A), int(B)]) * base
    for j in range(0, n):
        M[int(xp[j]) - 1, int(yp[j]) - 1] = FVec[j]
    zp = np.array([xp, yp])
    dup ={}
    # find duplicate
    for i in range(len(zp[0, :])):
        for j in range(i + 1, len(zp[0])):
            if zp[0, i] == zp[0, j] and zp[1, i] == zp[1, j]:
                #if i in dup.keys():
                #print("duplicate:" + str(i) + " " + str(j)+ "value: ")
                #dup.add(i)
                #dup[i].add(j)
                dup.setdefault(str(zp[0, i])+"-"+str(zp[1, i]), {i}).add(j)
    #print("Collisioni:"+str(len(dup.keys())))
    for index in dup.keys():
        x,y=index.split("-")
        M[int(float(x)) - 1, int(float(y)) - 1] = sum(FVec[list(dup[index])])/len(dup[index])
    return M


## Min Rect

In [0]:
import numpy as np
from scipy.spatial import ConvexHull

def minimum_bounding_rectangle(points):
    """
    Find the smallest bounding rectangle for a set of points.
    Returns a set of points representing the corners of the bounding box.

    :param points: an nx2 matrix of coordinates
    :rval: an nx2 matrix of coordinates
    """
    from scipy.ndimage.interpolation import rotate
    pi2 = np.pi/2.

    # get the convex hull for the points
    hull_points = points[ConvexHull(points).vertices]

    # calculate edge angles
    edges = np.zeros((len(hull_points)-1, 2))
    edges = hull_points[1:] - hull_points[:-1]

    angles = np.zeros((len(edges)))
    angles = np.arctan2(edges[:, 1], edges[:, 0])

    angles = np.abs(np.mod(angles, pi2))
    angles = np.unique(angles)

    # find rotation matrices
    # XXX both work
    rotations = np.vstack([
        np.cos(angles),
        np.cos(angles-pi2),
        np.cos(angles+pi2),
        np.cos(angles)]).T
#     rotations = np.vstack([
#         np.cos(angles),
#         -np.sin(angles),
#         np.sin(angles),
#         np.cos(angles)]).T
    rotations = rotations.reshape((-1, 2, 2))

    # apply rotations to the hull
    rot_points = np.dot(rotations, hull_points.T)

    # find the bounding points
    min_x = np.nanmin(rot_points[:, 0], axis=1)
    max_x = np.nanmax(rot_points[:, 0], axis=1)
    min_y = np.nanmin(rot_points[:, 1], axis=1)
    max_y = np.nanmax(rot_points[:, 1], axis=1)

    # find the box with the best area
    areas = (max_x - min_x) * (max_y - min_y)
    best_idx = np.argmin(areas)

    # return the best box
    x1 = max_x[best_idx]
    x2 = min_x[best_idx]
    y1 = max_y[best_idx]
    y2 = min_y[best_idx]
    r = rotations[best_idx]

    rval = np.zeros((4, 2))
    rval[0] = np.dot([x1, y2], r)
    rval[1] = np.dot([x2, y2], r)
    rval[2] = np.dot([x2, y1], r)
    rval[3] = np.dot([x1, y1], r)

    return rval

## Neural Network

In [0]:
import numpy as np
from keras import Model, Input
from keras.callbacks import EarlyStopping
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Activation, AveragePooling2D, Add, \
    Concatenate
from keras.optimizers import SGD, Adam
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split


def deep_train(images, y, param=None):
    print(param)
    x_train, x_test, y_train, y_test = train_test_split(images,
                                                        y,
                                                        test_size=0.2,
                                                        stratify=y,
                                                        random_state=100)
    x_train = np.array(x_train)
    x_test = np.array(x_test)

    image_size = x_train.shape[1]
    x_train = np.reshape(x_train, [-1, image_size, image_size, 1])
    x_test = np.reshape(x_test, [-1, image_size, image_size, 1])

    # num_filters = param["filter"]
    # num_filters2 = param["filter2"]

    num_filters = 2
    num_filters2 = 18

    kernel = param["kernel"]
    
    inputs = Input(shape=(image_size, image_size, 1))

    out = Conv2D(filters=num_filters,
                kernel_size=(kernel, kernel),
                padding="same")(inputs)
    out = BatchNormalization()(out)
    out = Activation('relu')(out)
    out = MaxPooling2D(strides=2, pool_size=2)(out)

    out = Conv2D(filters=2 * num_filters,
                kernel_size=(kernel, kernel),
                padding="same")(out)
    out = BatchNormalization()(out)
    out = Activation('relu')(out)
    out = MaxPooling2D(strides=2, pool_size=2)(out)

    out = Conv2D(filters=4 * num_filters,
                kernel_size=(kernel, kernel),
                padding="same")(out)
    out = BatchNormalization()(out)
    out = Activation('relu')(out)
    out = MaxPooling2D(strides=2, pool_size=2)(out)

    out = Conv2D(filters=8 * num_filters,
                kernel_size=(kernel, kernel),
                padding="same")(out)
    out = BatchNormalization()(out)
    out = Activation('relu')(out)

    # layer 2
    out2 = Conv2D(filters=num_filters2,
                  kernel_size=(kernel, kernel),
                  padding="same")(inputs)
    out2 = BatchNormalization()(out2)
    out2 = Activation('relu')(out2)
    out2 = MaxPooling2D(strides=2, pool_size=2)(out2)

    out2 = Conv2D(filters=2 * num_filters2,
                  kernel_size=(kernel, kernel),
                  padding="same")(out2)
    out2 = BatchNormalization()(out2)
    out2 = Activation('relu')(out2)
    out2 = MaxPooling2D(strides=2, pool_size=2)(out2)

    out2 = Conv2D(filters=4 * num_filters2,
                  kernel_size=(kernel, kernel),
                  padding="same")(out2)
    out2 = BatchNormalization()(out2)
    out2 = Activation('relu')(out2)
    out2 = MaxPooling2D(strides=2, pool_size=2)(out2)

    out2 = Conv2D(filters=8 * num_filters2,
                  kernel_size=(kernel, kernel),
                  padding="same")(out2)
    out2 = BatchNormalization()(out2)
    out2 = Activation('relu')(out2)
    # final layer
    outf = Concatenate()([out, out2])
    out_f = AveragePooling2D(strides=2, pool_size=2)(outf)
    out_f = Flatten()(out_f)
    predictions = Dense(2, activation='softmax')(out_f)

    # This creates a model that includes
    # the Input layer and three Dense layers
    model = Model(inputs=inputs, outputs=predictions)

    adam = Adam(lr=param["learning_rate"])

    # Compile the model.
    model.compile(
        optimizer=adam,
        loss='categorical_crossentropy',
        metrics=['accuracy'],
    )

    # Train the model.
    hist = model.fit(
        x_train,
        y_train,
        epochs=40,
        verbose=2,
        validation_data=(x_test, y_test),
        batch_size=param["batch"],
        callbacks=[EarlyStopping(monitor='val_acc', mode='max', patience=10)]

    )

      # score = hist.history["accuracy"][-1]
      # print("Accuracy: " + str(100.0 * score))

    return model


## Train

In [0]:
import csv
import json
import pickle
import timeit

import numpy as np
from hyperopt import STATUS_OK
from hyperopt import tpe, hp, Trials, fmin
from keras import backend as K
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, balanced_accuracy_score

XGlobal = []
YGlobal = []

XTestGlobal = []
YTestGlobal = []

SavedParameters = []


def hyperopt_fcn(params):
    global SavedParameters
    start = timeit.timeit()
    # for p in SavedParameters:
    #     if p["filter"] == params["filter"] and p["filter2"] == params["filter2"] and p["kernel"] == params["kernel"] and\
    #             p["learning_rate"] == params["learning_rate"] and p["momentum"] == params["momentum"]:
    #         return {'loss': np.inf, 'status': STATUS_OK}
    print("start train")
    model = deep_train(XGlobal, YGlobal, params)
    print("start predict")

    Y_predicted = model.predict(XTestGlobal, verbose=0, use_multiprocessing=True, workers=12)
    Y_predicted = np.argmax(Y_predicted, axis=1)
    end = timeit.timeit()

    cf = confusion_matrix(YTestGlobal, Y_predicted)
    print(cf)
    print(balanced_accuracy_score(YTestGlobal, Y_predicted))
    K.clear_session()
    # SavedParameters.append(
    #     {"balanced_accuracy": balanced_accuracy_score(YTestGlobal, Y_predicted) * 100, "TN": cf[0][0],
    #      "FP": cf[0][1], "FN": cf[1][0], "TP": cf[1][1], "filter": params["filter"], "filter2": params["filter2"],
    #      "kernel": params["kernel"], "learning_rate": params["learning_rate"], "momentum": params["momentum"]})
    SavedParameters.append(
        {"balanced_accuracy": balanced_accuracy_score(YTestGlobal, Y_predicted) * 100, "TN": cf[0][0],
         "FP": cf[0][1], "FN": cf[1][0], "TP": cf[1][1], "kernel": params["kernel"],
         "learning_rate": params["learning_rate"],
         "batch": params["batch"], "time": end - start
         })

    try:
        with open("/content/drive/My Drive/Tesi/param.csv", 'w', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=SavedParameters[0].keys())
            writer.writeheader()
            writer.writerows(SavedParameters)
    except IOError:
        print("I/O error")

    return {'loss': -balanced_accuracy_score(YTestGlobal, Y_predicted), 'status': STATUS_OK}


def train_norm(param, dataset, norm):
    np.random.seed(param["seed"])
    print("modelling dataset")
    global YGlobal
    YGlobal = to_categorical(dataset["Classification"])
    del dataset["Classification"]
    global YTestGlobal
    YTestGlobal = to_categorical(dataset["Ytest"])
    del dataset["Ytest"]

    global XGlobal
    global XTestGlobal

    if not param["LoadFromJson"]:
        # norm
        Out = {}
        if norm:
            print('NORM Min-Max')
            Out["Max"] = float(dataset["Xtrain"].max().max())
            Out["Min"] = float(dataset["Xtrain"].min().min())
            # NORM
            dataset["Xtrain"] = (dataset["Xtrain"] - Out["Min"]) / (Out["Max"] - Out["Min"])
            dataset["Xtrain"] = dataset["Xtrain"].fillna(0)

        # TODO implement norm 2
        print("trasposing")

        q = {"data": np.array(dataset["Xtrain"].values).transpose(), "method": param["Metod"],
             "max_px_size": param["Max_P_Size"]}
        print(q["method"])
        print(q["max_px_size"])

        # generate images
        XGlobal, image_model = Cart2Pixel(q, q["max_px_size"], q["max_px_size"], param["Dynamic_Size"])
        del q
        print("Train Images done!")
        # generate testingset image
        dataset["Xtest"] = np.array(dataset["Xtest"]).transpose()
        print("generating Test Images")
        print(dataset["Xtest"].shape)

        XTestGlobal = [ConvPixel(dataset["Xtest"][:, i], np.array(image_model["xp"]), np.array(image_model["yp"]),
                                 image_model["A"], image_model["B"]) for i in range(0, dataset["Xtest"].shape[1])]
        print("Test Images done!")

        del dataset["Xtest"]

        # saving testingset
        filename = "/content/drive/My Drive/Tesi/testingsetImage.pickle"
        f_myfile = open(filename, 'wb')
        pickle.dump(XTestGlobal, f_myfile)
        f_myfile.close()

    else:
        XGlobal = dataset["Xtrain"]
        XTestGlobal = dataset["Xtest"]
    del dataset["Xtrain"]
    del dataset["Xtest"]

    XTestGlobal = np.array(XTestGlobal)
    image_size = XTestGlobal.shape[1]
    XTestGlobal = np.reshape(XTestGlobal, [-1, image_size, image_size, 1])
    YTestGlobal = np.argmax(YTestGlobal, axis=1)

    # optimizable_variable = {"filter_size": 3, "kernel": 2, "filter_size2": 6,"learning_rate":1e-5,"momentum":0.8}
    optimizable_variable = {"kernel": hp.choice("kernel", np.arange(2, 16 + 1)),
                            "batch": hp.choice("batch", [64, 128, 256, 512]),
                            "learning_rate": hp.uniform("learning_rate", 0.0001, 0.01)}

    trials = Trials()
    global SavedParameters
    SavedParameters150 = []
    fmin(hyperopt_fcn, optimizable_variable, trials=trials, algo=tpe.suggest, max_evals=20)

    print("migliori parametri")
    SavedParameters = sorted(SavedParameters, key=lambda i: i['balanced_accuracy'], reverse=True)
    print(SavedParameters[0])

    # returning best model with hyperopt parameters
    model = deep_train(XGlobal, YGlobal, SavedParameters[0])
    print("accuracy" + str(model))
    return model


## Load data from drive

In [0]:
param = {"Max_P_Size": 33, "Dynamic_Size": False, 'Metod': 'tSNE', "ValidRatio": 0.1, "seed": 180, "Mode": "neural",
         "LoadFromJson": True}

In [0]:
import pandas as pd
import csv
import numpy as np
import pickle
from google.colab import drive
drive.mount('/content/drive')

if not param["LoadFromJson"]:
  with open('/content/drive/My Drive/Tesi/TrainOneCls.csv', 'r') as file:
    data = {"Xtrain": pd.DataFrame(list(csv.DictReader(file))).astype(float), "class": 2}
    print(data["Xtrain"].shape)
    data["Classification"] = data["Xtrain"]["Classification"]
    del data["Xtrain"]["Classification"]
  print("train loaded")

  with open('/content/drive/My Drive/Tesi/Testing.csv', 'r') as file:
    data["Xtest"]=pd.DataFrame(list(csv.DictReader(file))).astype(float)
    data["Ytest"] = data["Xtest"]["Classification"]
    del data["Xtest"]["Classification"]
    del data["Xtest"]['']
  print("test loaded")

  
else:
    images={}
    f_myfile = open('/content/drive/My Drive/Tesi/trainingsetImage.pickle', 'rb')
    images["Xtrain"] = pickle.load(f_myfile)
    f_myfile.close()


    f_myfile = open('/content/drive/My Drive/Tesi/y_trainingset.pickle', 'rb')
    images["Classification"] = pickle.load(f_myfile)
    f_myfile.close()

    f_myfile = open('/content/drive/My Drive/Tesi/testingsetImage.pickle', 'rb')
    images["Xtest"] = pickle.load(f_myfile)
    f_myfile.close()

    f_myfile = open('/content/drive/My Drive/Tesi/y_testingset.pickle', 'rb')
    images["Ytest"] = pickle.load(f_myfile)
    f_myfile.close()



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import csv
import numpy as np
import pickle
from google.colab import drive
drive.mount('/content/drive')

if not param["LoadFromJson"]:
  with open('/content/drive/My Drive/Tesi/TrainOneCls.csv', 'r') as file:
    data = {"Xtrain": pd.DataFrame(list(csv.DictReader(file))).astype(float), "class": 2}
    print(data["Xtrain"].shape)
    data["Classification"] = data["Xtrain"]["Classification"]
    del data["Xtrain"]["Classification"]
  print("train loaded")

  with open('/content/drive/My Drive/Tesi/Testing.csv', 'r') as file:
    data["Xtest"]=pd.DataFrame(list(csv.DictReader(file))).astype(float)
    data["Ytest"] = data["Xtest"]["Classification"]
    del data["Xtest"]["Classification"]
    del data["Xtest"]['']

  filename = "/content/drive/My Drive/Tesi/y_testingset.pickle"
  f_myfile = open(filename, 'wb')
  pickle.dump(data["Ytest"], f_myfile)
  f_myfile.close()

  print("test loaded")
else:
    images={}
    f_myfile = open('/content/drive/My Drive/Tesi/trainingsetImage.pickle', 'rb')
    images["Xtrain"] = pickle.load(f_myfile)
    f_myfile.close()


    f_myfile = open('/content/drive/My Drive/Tesi/y_trainingset.pickle', 'rb')
    images["Classification"] = pickle.load(f_myfile)
    f_myfile.close()

    f_myfile = open('/content/drive/My Drive/Tesi/testingsetImage.pickle', 'rb')
    images["Xtest"] = pickle.load(f_myfile)
    f_myfile.close()

    f_myfile = open('/content/drive/My Drive/Tesi/y_testingset.pickle', 'rb')
    images["Ytest"] = pickle.load(f_myfile)
    f_myfile.close()



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Run


In [0]:
model = train_norm(param, images, norm=False)